# IBM Data Science Capstone Project Notebook

This is my attempt at doing the IBM Capstone Project from "IBM Data Science Professional Certificate". 

# Assignment II - Clustering and Segmenting Toronto Neighbourhoods

The data can be found in the following link: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

Part of the challenge is to learn how to get data from other media, in this case wikipedia.

## Necessary Libraries

In [1]:
# data preparation

import urllib.request
try: 
    from BeautifulSoup import BeautifulSoup
except ImportError:
    from bs4 import BeautifulSoup
import pandas as pd
!pip install geocoder #if not present
import geocoder

## Data Preparation

In [2]:
with urllib.request.urlopen('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M') as response:
   html = response.read()

html
#can we parse this with json?

b'<!DOCTYPE html>\n<html class="client-nojs" lang="en" dir="ltr">\n<head>\n<meta charset="UTF-8"/>\n<title>List of postal codes of Canada: M - Wikipedia</title>\n<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"4b006a0d-348a-4cac-9719-e716c298f0e4","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":977312452,"wgRevisionId":977312452,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Onta

In [3]:
#parsing the HTML

parsed_html = BeautifulSoup(html)

html_text = parsed_html.body.find('table', attrs={'class':'wikitable sortable'}).text

components = html_text.split('\n') #split by line

components = list(filter(lambda a: a != '', components)) #remove all blank spaces


components

['Postal Code',
 'Community',
 'Neighbourhood',
 'M1A',
 'Not assigned',
 'Not assigned',
 'M2A',
 'Not assigned',
 'Not assigned',
 'M3A',
 'North York',
 'Parkwoods',
 'M4A',
 'North York',
 'Victoria Village',
 'M5A',
 'Downtown Toronto',
 'Regent Park, Harbourfront',
 'M6A',
 'North York',
 'Lawrence Manor, Lawrence Heights',
 'M7A',
 'Downtown Toronto',
 "Queen's Park, Ontario Provincial Government",
 'M8A',
 'Not assigned',
 'Not assigned',
 'M9A',
 'Etobicoke',
 'Islington Avenue, Humber Valley Village',
 'M1B',
 'Scarborough',
 'Malvern, Rouge',
 'M2B',
 'Not assigned',
 'Not assigned',
 'M3B',
 'North York',
 'Don Mills',
 'M4B',
 'East York',
 'Parkview Hill, Woodbine Gardens',
 'M5B',
 'Downtown Toronto',
 'Garden District, Ryerson',
 'M6B',
 'North York',
 'Glencairn',
 'M7B',
 'Not assigned',
 'Not assigned',
 'M8B',
 'Not assigned',
 'Not assigned',
 'M9B',
 'Etobicoke',
 'West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale',
 'M1C',
 'Scarborough',
 'R

In [11]:
#every 3 items are a data point...

columns = []
post = []
borough = []
neighbor = []

for index,item in enumerate(components):
    if (index<3):
        columns.append(item)
    elif ((index+1)%3==0): #at every third item, which marks the end of a 'row' in the df, add the values!
        post.append(components[index-2])
        borough.append(components[index-1])
        neighbor.append(item)

data = [post,borough,neighbor]
data = { col_name:col_data for col_name,col_data in zip(columns,data)}

df = pd.DataFrame(data=data)
df.columns = ['Postal Code','Borough','Neighbourhood']
df.head()

#datafram created!

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


## Data Processing

In [12]:
df = df[df['Borough'] != 'Not assigned']

print(df[df['Neighbourhood'] == 'Not assigned'])

df['Neighbourhood'].loc[df['Neighbourhood']=='Not assigned'] = df['Borough'].loc[df['Neighbourhood']=='Not assigned']

print('Shape of the datafram is {}'.format(df.shape))
df.head()

Empty DataFrame
Columns: [Postal Code, Borough, Neighbourhood]
Index: []
Shape of the datafram is (103, 3)


,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Extracting Geolocation data

In [46]:
""" Note: this portion of the notebook does not work for some reason... continue to the next cell """

coordinates = None
latitude = []
longitude = []
counter = 1; # added to determine the completion rate of the geolocator code!

for post_code in df['Postal Code'].values:
    while (coordinates == None):
        g = geocoder.google('{}, Toronto, Ontario'.format(post_code))
        coordinates = g.latlng
    print(counter)
    counter += counter
    latitude.append(coordinates[0])
    longitude.apppend(coordinates[1])

df['Latitude'] = latitude
df['Longitude'] = longitude

df.head()

KeyboardInterrupt: 

In [13]:
!wget -O 'geo_data.csv' https://cocl.us/Geospatial_data

--2020-09-12 19:11:40--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.55.161.7
Connecting to cocl.us (cocl.us)|169.55.161.7|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-09-12 19:11:42--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 185.235.236.197
Connecting to ibm.box.com (ibm.box.com)|185.235.236.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-09-12 19:11:42--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-09-12 19

In [14]:
geo_data = pd.read_csv('geo_data.csv')
geo_data.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [16]:
latitude = []
longitude = []

for post_code in df['Postal Code'].values:
    latitude.append(geo_data.Latitude[geo_data['Postal Code']==post_code].values[0])
    longitude.append(geo_data.Longitude[geo_data['Postal Code']==post_code].values[0])

df['Latitude'] = latitude
df['Longitude'] = longitude

df.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
2,M3A,North York,Parkwoods,43.753259,-79.329656
3,M4A,North York,Victoria Village,43.725882,-79.315572
4,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
5,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


# Sources

- https://stackoverflow.com/questions/11709079/parsing-html-using-python
- https://stackoverflow.com/questions/38116305/read-output-of-php-file-using-python